### Load all feature values until 2100 + dataframe for all PAs

In [2]:
import pandas as pd
import os

In [3]:
df_merged_eco_variables = pd.read_csv('../raw_data/final_merge/eco_variables_until_2100.csv')
df_merged_eco_variables

,Unnamed: 0,latitude,longitude,quarter_start,temperature_quarterly_mean,precipitation_quarterly_mean,water-vapor-pressure_quarterly_mean,cloud-cover_quarterly_mean
0,0,27.75,-32.25,2015-01-01,19.56,2.516969e-08,958.00810,0.550595
1,1,27.75,-32.25,2015-04-01,20.83,7.676677e-09,1019.88770,0.568784
2,2,27.75,-32.25,2015-07-01,23.49,7.344711e-09,1111.47960,0.452135
3,3,27.75,-32.25,2015-10-01,22.31,2.013239e-08,1061.99770,0.569084
4,4,27.75,-32.25,2016-01-01,20.02,1.796181e-08,986.56890,0.604120
...,...,...,...,...,...,...,...,...
3888915,3888915,69.75,33.75,2099-10-01,4.96,4.002802e-08,665.55804,0.882963
3888916,3888916,69.75,33.75,2100-01-01,3.18,2.680442e-08,589.26170,0.861919
3888917,3888917,69.75,33.75,2100-04-01,4.65,2.021687e-08,638.46564,0.808334
3888918,3888918,69.75,33.75,2100-07-01,8.78,2.884796e-08,783.57196,0.832920


In [4]:
final_table_2015_2024_multiple_pa = pd.read_csv('../raw_data/final_merge/final_lc_multiple_areas_2015_2024.csv')
final_table_2015_2024_multiple_pa

,date,SITECODE,LC,LC_PROPORTION,lon,lat
0,2015-07-01,IT5320009,Water,3.587173,13.292268,43.532192
1,2015-07-01,IT5320009,Trees,25.068529,13.292268,43.532192
2,2015-07-01,IT5320009,Grass,3.445941,13.292268,43.532192
3,2015-07-01,IT5320009,Flooded Vegetation,1.554780,13.292268,43.532192
4,2015-07-01,IT5320009,Crops,36.791729,13.292268,43.532192
...,...,...,...,...,...,...
228424,2024-04-01,IT4020021,Crops,36.639650,10.156949,44.744558
228425,2024-04-01,IT4020021,Shrub and Scrub,3.665165,10.156949,44.744558
228426,2024-04-01,IT4020021,Built Area,6.063098,10.156949,44.744558
228427,2024-04-01,IT4020021,Bare Ground,13.193696,10.156949,44.744558


In [6]:
from scipy.spatial import distance_matrix
import pandas as pd
from datetime import datetime
import numpy as np

# Ensure datetime columns are in datetime format
df_merged_eco_variables['quarter_start'] = pd.to_datetime(df_merged_eco_variables['quarter_start'])
final_table_2015_2024_multiple_pa['date'] = pd.to_datetime(final_table_2015_2024_multiple_pa['date'])

# Filter the df_merged_eco_variables for dates starting from 2025
future_eco_variables = df_merged_eco_variables[df_merged_eco_variables['quarter_start'] >= '2025-01-01']

# Get the unique protected area coordinates
protected_area_coords = final_table_2015_2024_multiple_pa[['SITECODE', 'lon', 'lat']].drop_duplicates()

# Initialize the result DataFrame
future_values_2100_all_pa = pd.DataFrame()

# Iterate through each quarter from 2025 to 2100
quarters = pd.date_range(start='2025-01-01', end='2100-10-01', freq='QS')

for quarter in quarters:
    # Filter future_eco_variables for the current quarter
    df_current_quarter = future_eco_variables[future_eco_variables['quarter_start'] == quarter]

    # Calculate distance matrix
    distances = distance_matrix(protected_area_coords[['lat', 'lon']], df_current_quarter[['latitude', 'longitude']])

    # Find the index of the nearest coordinates in df_current_quarter for each protected area
    nearest_idx = np.argmin(distances, axis=1)

    # Retrieve feature values for the nearest coordinates
    temperature_values = df_current_quarter.iloc[nearest_idx]['temperature_quarterly_mean'].values
    precipitation_values = df_current_quarter.iloc[nearest_idx]['precipitation_quarterly_mean'].values
    water_values = df_current_quarter.iloc[nearest_idx]['water-vapor-pressure_quarterly_mean'].values
    cloud_values = df_current_quarter.iloc[nearest_idx]['cloud-cover_quarterly_mean'].values

    # Create a temporary DataFrame to hold the results for this quarter
    temp_df = protected_area_coords.copy()
    temp_df['quarter_start'] = quarter
    temp_df['temperature_quarterly_mean'] = temperature_values
    temp_df['precipitation_quarterly_mean'] = precipitation_values
    temp_df['water-vapor-pressure_quarterly_mean'] = water_values
    temp_df['cloud-cover_quarterly_mean'] = cloud_values

    # Append the results for this quarter to the result DataFrame
    future_values_2100_all_pa = pd.concat([future_values_2100_all_pa, temp_df], ignore_index=True)

# Ensure the 'quarter_start' column is datetime
future_values_2100_all_pa['quarter_start'] = pd.to_datetime(future_values_2100_all_pa['quarter_start'])


In [7]:
future_values_2100_all_pa

,SITECODE,lon,lat,quarter_start,temperature_quarterly_mean,precipitation_quarterly_mean,water-vapor-pressure_quarterly_mean,cloud-cover_quarterly_mean
0,IT5320009,13.292268,43.532192,2025-01-01,8.65,1.178506e-08,650.85570,0.500273
1,IT4050014,11.128795,44.319651,2025-01-01,5.11,1.629546e-08,606.74290,0.544266
2,IT4060015,12.256245,44.849636,2025-01-01,7.75,9.604958e-09,666.82480,0.515485
3,IT5330008,13.132352,42.878579,2025-01-01,2.14,2.546247e-08,538.16170,0.572809
4,IT4080001,11.801841,43.847954,2025-01-01,4.39,2.139190e-08,592.76886,0.588862
...,...,...,...,...,...,...,...,...
200027,IT4050013,11.092391,44.210584,2100-10-01,10.01,3.722503e-08,774.13670,0.603561
200028,IT4030002,10.292255,44.385383,2100-10-01,8.32,5.366960e-08,749.49335,0.646030
200029,IT4030003,10.276491,44.308203,2100-10-01,8.32,5.366960e-08,749.49335,0.646030
200030,IT4020017,10.318769,44.933736,2100-10-01,10.46,3.412798e-08,781.61035,0.575535


In [8]:
future_values_2100_all_pa.SITECODE.nunique()

624

In [9]:
future_values_2100_all_pa.quarter_start.nunique()

304

In [10]:
future_values_2100_all_pa = future_values_2100_all_pa.drop_duplicates()
future_values_2100_all_pa

,SITECODE,lon,lat,quarter_start,temperature_quarterly_mean,precipitation_quarterly_mean,water-vapor-pressure_quarterly_mean,cloud-cover_quarterly_mean
0,IT5320009,13.292268,43.532192,2025-01-01,8.65,1.178506e-08,650.85570,0.500273
1,IT4050014,11.128795,44.319651,2025-01-01,5.11,1.629546e-08,606.74290,0.544266
2,IT4060015,12.256245,44.849636,2025-01-01,7.75,9.604958e-09,666.82480,0.515485
3,IT5330008,13.132352,42.878579,2025-01-01,2.14,2.546247e-08,538.16170,0.572809
4,IT4080001,11.801841,43.847954,2025-01-01,4.39,2.139190e-08,592.76886,0.588862
...,...,...,...,...,...,...,...,...
200027,IT4050013,11.092391,44.210584,2100-10-01,10.01,3.722503e-08,774.13670,0.603561
200028,IT4030002,10.292255,44.385383,2100-10-01,8.32,5.366960e-08,749.49335,0.646030
200029,IT4030003,10.276491,44.308203,2100-10-01,8.32,5.366960e-08,749.49335,0.646030
200030,IT4020017,10.318769,44.933736,2100-10-01,10.46,3.412798e-08,781.61035,0.575535


In [11]:
future_values_2100_all_pa.to_csv('../raw_data/final_merge/all_pa_future_values_2100.csv', index=False)